<a href="https://colab.research.google.com/github/jeffersonramelo/Paper-pos-doc/blob/main/modelos_tradicionais.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd

# Caminho para o arquivo XLSX
caminho_arquivo = '/content/2010.xlsx'

# Lê o arquivo XLSX
dados = pd.read_excel(caminho_arquivo)

# Agora, 'dados' contém os dados do arquivo XLSX


In [8]:
import pandas as pd

# Supondo que seu DataFrame se chame "dados"

# Listar todas as colunas do DataFrame "dados"
print(dados.columns)


Index(['id', 'COD', 'SETOR', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8',
       'd9', 'VM30Abril11', 'PL10', 'LL10', 'AT10', 'PT10', 'INTANG10',
       'CAXEEQUIV10', 'GREC10', 'REC09', 'REC10', 'CAXOP10', 'ORA10'],
      dtype='object')


tratar valores ausentes

In [43]:
import pandas as pd
import numpy as np

# Supondo que seu DataFrame se chame "dados"

# Substituir apenas traços ("-") e valores em branco por NaN
dados = dados.replace(['-',''], np.nan)

# Calcular a média de todas as colunas, ignorando os valores NaN
media_por_coluna = dados.mean(numeric_only=True)

# Preencher os valores ausentes em todas as colunas com a média de cada coluna
dados.fillna(media_por_coluna, inplace=True)


In [11]:
# Visualizar o conteúdo do DataFrame "dados"
print(dados)

      id     COD              SETOR  d1  d2  d3  d4  d5  d6  d7  ...  \
0      4   ABCB4         Financeiro   0   0   0   1   0   0   0  ...   
1      5   EALT4   Bens industriais   1   0   0   0   0   0   0  ...   
2      9   AFLT3  Utilidade pública   0   0   0   0   0   0   0  ...   
3     12  BRGE12         Financeiro   0   0   0   1   0   0   0  ...   
4     13   CRIV4         Financeiro   0   0   0   1   0   0   0  ...   
..   ...     ...                ...  ..  ..  ..  ..  ..  ..  ..  ...   
228  388   MWET4   Bens industriais   1   0   0   0   0   0   0  ...   
229  389   WHRL4    Consumo cíclico   0   1   0   0   0   0   0  ...   
230  390   PORT3   Bens industriais   1   0   0   0   0   0   0  ...   
231  392   WLMM4   Bens industriais   1   0   0   0   0   0   0  ...   
232  394   YDUQ3    Consumo cíclico   0   1   0   0   0   0   0  ...   

         LL10       AT10       PT10      INTANG10    CAXEEQUIV10    GREC10  \
0    202224.0  9816888.0  8468970.0  1.270539e+06  776299

REGRESSÃO SEM ESCALA

tratar outliers: A winsorização ao nível de 1% envolve substituir os 1% dos valores mais extremos por valores no limite do intervalo aceitável, ou seja, os 1% menores e os 1% maiores dos valores.

In [38]:
import pandas as pd
import numpy as np
from scipy.stats import mstats

# Supondo que seu DataFrame se chame "dados"

# Lista de variáveis para winsorizar
variaveis_winsorizar = ['VM30Abril11', 'PL10', 'LL10', 'AT10', 'PT10', 'INTANG10',
                        'CAXEEQUIV10', 'GREC10', 'REC09', 'REC10', 'CAXOP10', 'ORA10']

# Aplicar winsorização de 1% nas variáveis
for coluna in variaveis_winsorizar:
    dados[coluna] = mstats.winsorize(dados[coluna], limits=[0.01, 0.01])

# Agora, as variáveis especificadas foram winsorizadas


In [39]:
import statsmodels.api as sm

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PL10', 'LL10', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'INTANG10',
       'CAXEEQUIV10', 'GREC10', 'REC10', 'CAXOP10', 'ORA10']]
y = dados['VM30Abril11']

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Visualize os resultados do modelo
print(modelo.summary())

# Calcule o MedAE
y_pred = modelo.predict(X)
medae = np.median(np.abs(y - y_pred))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço)
medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE):", medae)
print("MedAE escalado pelo valor (ou preço) em percentagem:", medae_scaled)


                            OLS Regression Results                            
Dep. Variable:            VM30Abril11   R-squared:                       0.945
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                     216.9
Date:                Wed, 20 Sep 2023   Prob (F-statistic):          4.98e-125
Time:                        00:35:34   Log-Likelihood:                -3938.9
No. Observations:                 233   AIC:                             7914.
Df Residuals:                     215   BIC:                             7976.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -7.131e+05   2.29e+06     -0.311      

Em relação aos resultados apresentados no seu modelo de regressão:

O valor do teste de Durbin-Watson é 1.676, o que sugere uma leve autocorrelação positiva nos resíduos, mas não muito forte.
O valor-p associado ao teste de Jarque-Bera é praticamente zero (0.00), o que indica que os resíduos não seguem uma distribuição normal.

Interpretação DW:

Valor DW ≈ 2: Ausência de autocorrelação significativa nos resíduos (boa notícia).
Valor DW < 2: Autocorrelação positiva nos resíduos (erros adjacentes são correlacionados positivamente).
Valor DW > 2: Autocorrelação negativa nos resíduos (erros adjacentes são correlacionados negativamente).

Nesse caso específico, o valor de 17.74% indica que, em média, as previsões do modelo estão erradas em cerca de 17.74% do valor médio (ou preço) da variável de resposta. Isso ajuda a ter uma ideia do tamanho relativo dos erros do modelo em relação ao valor médio dos dados.

teste de heterocedasticidade de White. Se o valor-p for menor que um nível de significância escolhido (por exemplo, 0,05), você pode rejeitar a hipótese nula de homocedasticidade

In [40]:
import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_white

# Calcule os resíduos do modelo
residuos = modelo.resid

# Realize o teste de White para heterocedasticidade
teste_white = het_white(residuos, X)

# Imprima os resultados do teste
print("Estatística do teste de White:", teste_white[0])
print("Valor-p do teste de White:", teste_white[1])

AssertionError: ignored

In [35]:
import pandas as pd

# Caminho para o arquivo XLSX
caminho_arquivo = '/content/2010.xlsx'

# Lê o arquivo XLSX
dados = pd.read_excel(caminho_arquivo)

# Agora, 'dados' contém os dados do arquivo XLSX

minimos quadrados ponderados (MQP), com pesos com base na variância dos erros

In [44]:
import statsmodels.api as sm
import numpy as np

# Suponha que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PL10', 'LL10', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'INTANG10',
       'CAXEEQUIV10', 'GREC10', 'REC10', 'CAXOP10', 'ORA10']]
y = dados['VM30Abril11']

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão linear
modelo = sm.OLS(y, X).fit()

# Calcule os resíduos (erros)
residuos = modelo.resid

# Calcule a variância dos resíduos
variancia_residuos = np.var(residuos)

# Calcule os pesos com base na variância dos resíduos (inverso da variância)
pesos = 1 / variancia_residuos

# Crie o modelo de regressão linear ponderada (WLS) com os pesos calculados
modelo_wls = sm.WLS(y, X, weights=pesos).fit()

# Visualize os resultados do modelo ponderado
print(modelo_wls.summary())

# Calcule os resíduos ponderados (erros ponderados)
y_pred_wls = modelo_wls.predict(X)
residuos_wls = y - y_pred_wls

# Calcule o MedAE ponderado
medae_wls = np.median(np.abs(residuos_wls))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço) em percentagem
medae_scaled_wls = (medae_wls / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE) ponderado:", medae_wls)
print("MedAE ponderado escalado pelo valor (ou preço) em percentagem:", medae_scaled_wls)


                            WLS Regression Results                            
Dep. Variable:            VM30Abril11   R-squared:                       0.979
Model:                            WLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                     587.8
Date:                Wed, 20 Sep 2023   Prob (F-statistic):          8.31e-170
Time:                        00:39:55   Log-Likelihood:                -3917.9
No. Observations:                 233   AIC:                             7872.
Df Residuals:                     215   BIC:                             7934.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -8.713e+05   2.12e+06     -0.412      

REGRESSÃO QUANTÍLICA

In [45]:
import statsmodels.api as sm

# Supondo que 'dados' seja o DataFrame com suas variáveis

# Defina as variáveis independentes e dependente
X = dados[['PL10', 'LL10', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'INTANG10',
       'CAXEEQUIV10', 'GREC10', 'REC10', 'CAXOP10', 'ORA10']]
y = dados['VM30Abril11']

# Adicione uma constante ao conjunto de variáveis independentes
X = sm.add_constant(X)

# Crie o modelo de regressão quantílica
modelo = sm.QuantReg(y, X).fit(q=0.5)  # q=0.5 representa a mediana, ajuste conforme necessário

# Visualize os resultados do modelo
print(modelo.summary())
# Faça previsões com o modelo quantílico
y_pred_quantreg = modelo.predict(X)

# Calcule o MedAE para o modelo quantílico
medae_quantreg = np.median(np.abs(y - y_pred_quantreg))

# Calcule o valor médio da variável de resposta
valor_medio_y = np.mean(y)

# Calcule o MedAE escalado pelo valor (ou preço) em percentagem
medae_scaled_quantreg = (medae_quantreg / valor_medio_y) * 100  # Em percentagem

print("Median Absolute Error (MedAE) quantílico:", medae_quantreg)
print("MedAE quantílico escalado pelo valor (ou preço) em percentagem:", medae_scaled_quantreg)


                         QuantReg Regression Results                          
Dep. Variable:            VM30Abril11   Pseudo R-squared:               0.7573
Model:                       QuantReg   Bandwidth:                   8.290e+05
Method:                 Least Squares   Sparsity:                    2.294e+06
Date:                Wed, 20 Sep 2023   No. Observations:                  233
Time:                        00:40:52   Df Residuals:                      215
                                        Df Model:                           17
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const       -5.685e-06   3.94e+04  -1.44e-10      1.000   -7.76e+04    7.76e+04
PL10            0.1758      0.016     10.956      0.000       0.144       0.207
LL10            4.9762      0.084     59.417      0.000       4.811       5.141
d1          -4.523e-12   6493.035  -6.97e-16   

ARVORE DE DECISÃO

In [46]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL10', 'LL10', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'INTANG10',
       'CAXEEQUIV10', 'GREC10', 'REC10', 'CAXOP10', 'ORA10']]
y = dados['VM30Abril11']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de árvore de decisão com os parâmetros desejados
    modelo_arvore = DecisionTreeRegressor(min_samples_split=5, random_state=42)

    # Criar o modelo Bagging com 500 árvores
    bagging_model = BaggingRegressor(base_estimator=modelo_arvore, n_estimators=500, bootstrap=True, random_state=42)

    bagging_model.fit(X_train, y_train)
    y_pred = bagging_model.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    # Visualizar a importância dos coeficientes (importância das variáveis)
    importancia_coeficientes = np.mean([tree.feature_importances_ for tree in bagging_model.estimators_], axis=0)
    print("Importância dos Coeficientes:", importancia_coeficientes)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Importância dos Coeficientes: [9.97888241e-02 5.48306044e-01 7.80348009e-05 2.27016493e-05
 1.18546452e-04 4.53173493e-05 2.23699173e-05 9.94562608e-04
 7.61760245e-05 2.90721675e-05 5.94548763e-05 3.88702476e-02
 4.71314322e-02 1.15705369e-03 6.94624883e-02 1.92407498e-01
 1.43017560e-03]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Importância dos Coeficientes: [8.41303038e-02 5.64070915e-01 8.02449944e-05 2.12786298e-05
 3.71584550e-04 2.01780792e-04 1.13311738e-03 1.11817239e-03
 7.04103783e-05 2.72379131e-05 9.33922892e-05 5.15163661e-02
 4.77149057e-02 3.11191292e-03 5.23976910e-02 1.93130002e-01
 8.10684672e-04]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


...

Importância dos Coeficientes: [7.58864501e-02 6.12355495e-01 1.41213219e-04 2.77633124e-05
 2.48552418e-04 7.96753651e-04 3.87456852e-03 1.46325501e-04
 1.04038760e-04 4.09171704e-05 1.93145089e-04 1.76697535e-03
 2.32218605e-02 3.92341118e-03 2.28096051e-02 2.52910153e-01
 1.55277272e-03]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Importância dos Coeficientes: [8.81862925e-02 5.69837152e-01 9.82374237e-05 2.58248593e-05
 1.40623381e-04 4.38105035e-04 1.45137023e-03 5.31757806e-04
 1.18593848e-04 3.19263821e-05 6.53133214e-05 2.30766483e-02
 7.79511193e-02 2.70876089e-03 4.74405665e-02 1.86657234e-01
 1.24047487e-03]


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(


Importância dos Coeficientes: [1.25273058e-01 3.94963516e-01 1.05015368e-04 3.59982735e-05
 8.45412702e-04 4.75362895e-04 1.06913045e-03 4.83579405e-05
 8.59506593e-05 4.17643638e-05 8.79723301e-05 6.69152766e-02
 1.06954791e-01 1.64752395e-03 1.35443903e-01 1.65445612e-01
 5.61354700e-04]
Média do MedAE escalado pelo valor (ou preço) em percentagem: 11.811472460411899
Média do R²: 0.6531190987067395


FLORESTA ALEATÓRIA

In [47]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL10', 'LL10', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'INTANG10',
       'CAXEEQUIV10', 'GREC10', 'REC10', 'CAXOP10', 'ORA10']]
y = dados['VM30Abril11']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de Floresta Aleatória com os parâmetros desejados
    random_forest_model = RandomForestRegressor(n_estimators=500, random_state=42)

    random_forest_model.fit(X_train, y_train)
    y_pred = random_forest_model.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    # Visualizar a importância das variáveis
    importancia_variaveis = random_forest_model.feature_importances_
    print("Importância das Variáveis:", importancia_variaveis)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


Importância das Variáveis: [9.45324502e-02 5.27509318e-01 7.75173969e-05 3.17505292e-05
 7.64787076e-04 3.67579030e-03 5.34448345e-03 4.34944304e-03
 7.98966196e-05 2.73970045e-05 6.90909199e-05 4.16200162e-02
 5.21939774e-02 9.34805217e-03 7.97539108e-02 1.75063196e-01
 5.55892304e-03]
Importância das Variáveis: [1.03121962e-01 5.40180289e-01 9.50792368e-05 2.79678489e-05
 5.65813139e-04 2.50659241e-03 4.87524265e-03 4.45028177e-03
 7.73104513e-05 2.62689266e-05 1.07006221e-04 5.04954037e-02
 5.09728876e-02 9.33333805e-03 5.77498584e-02 1.70151920e-01
 5.26277857e-03]
Importância das Variáveis: [9.32462499e-02 4.77214721e-01 1.55716858e-04 4.65547445e-05
 1.87980118e-03 3.14077751e-03 1.28876136e-04 8.32785835e-03
 1.55290987e-04 4.90024581e-05 1.28127108e-04 1.12659354e-01
 5.48770514e-02 4.69731354e-03 5.51732399e-02 1.80220298e-01
 7.89976569e-03]
Importância das Variáveis: [6.30805237e-02 5.91171914e-01 1.21640179e-04 2.50015272e-05
 1.01023155e-03 2.99186006e-03 4.06234456e-03 3.

GRADIENT BOOSTING

In [48]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL10', 'LL10', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'INTANG10',
       'CAXEEQUIV10', 'GREC10', 'REC10', 'CAXOP10', 'ORA10']]
y = dados['VM30Abril11']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de Regressão Gradient Boosting com os parâmetros desejados
    gradient_boosting_model = GradientBoostingRegressor(n_estimators=500, random_state=42)

    gradient_boosting_model.fit(X_train, y_train)
    y_pred = gradient_boosting_model.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

    # Visualizar a importância das variáveis
    importancia_variaveis = gradient_boosting_model.feature_importances_
    print("Importância das Variáveis:", importancia_variaveis)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


Importância das Variáveis: [3.06427477e-02 7.35181092e-01 8.38810638e-06 6.30734976e-06
 4.60366639e-04 1.93865189e-03 3.58324257e-04 3.95400755e-03
 6.03353005e-05 1.45156547e-05 7.87006752e-05 1.75397961e-03
 8.04373069e-02 8.96126355e-03 1.28111980e-02 1.13586221e-01
 9.74659385e-03]
Importância das Variáveis: [1.61682162e-02 7.50092632e-01 1.16238727e-05 2.62326392e-06
 1.62747243e-03 3.16783042e-03 3.94278181e-03 8.75080320e-03
 7.62886578e-05 4.05167744e-06 5.07637128e-05 2.59235658e-03
 2.70628089e-02 5.79270999e-04 2.94031476e-02 1.48633476e-01
 7.83385280e-03]
Importância das Variáveis: [2.66362076e-02 6.24098363e-01 1.31270736e-05 3.77033489e-06
 4.47053087e-04 5.09648012e-04 2.10510247e-05 8.98031352e-02
 1.07982918e-04 1.28162398e-05 6.72718523e-05 1.01115596e-03
 5.20385425e-02 1.26958126e-02 6.24117169e-03 1.71310229e-01
 1.49826620e-02]
Importância das Variáveis: [2.61929457e-02 8.80492749e-01 1.73767481e-05 2.25553690e-07
 7.46367428e-04 1.19332885e-04 1.37014278e-03 4.

REDES NEURAIS

In [49]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

# Supondo que 'dados' seja o DataFrame com suas variáveis
X = dados[['PL10', 'LL10', 'd1', 'd2', 'd3', 'd4', 'd5', 'd6', 'd7', 'd8', 'd9', 'INTANG10',
       'CAXEEQUIV10', 'GREC10', 'REC10', 'CAXOP10', 'ORA10']]
y = dados['VM30Abril11']

# Configurar a validação cruzada com 10 folds
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# Listas para armazenar resultados
medae_scores = []
r2_scores = []

# Treinar e avaliar o modelo dentro da amostra usando validação cruzada
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Criar o modelo de regressão de rede neural com os parâmetros desejados
    modelo_rede_neural = MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=1000, random_state=42)

    modelo_rede_neural.fit(X_train, y_train)
    y_pred = modelo_rede_neural.predict(X_test)

    # Calcular o MedAE escalado pelo valor (ou preço) em percentagem
    medae = np.median(np.abs(y_test - y_pred))
    valor_medio_y = np.mean(y_test)
    medae_scaled = (medae / valor_medio_y) * 100  # Em percentagem
    medae_scores.append(medae_scaled)

    # Calcular o R²
    r2 = r2_score(y_test, y_pred)
    r2_scores.append(r2)

# Média do MedAE escalado pelo valor (ou preço) em percentagem
mean_medae_scaled = np.mean(medae_scores)
print("Média do MedAE escalado pelo valor (ou preço) em percentagem:", mean_medae_scaled)

# Média do R²
mean_r2 = np.mean(r2_scores)
print("Média do R²:", mean_r2)


Média do MedAE escalado pelo valor (ou preço) em percentagem: 12.801316925478613
Média do R²: 0.7046287538694587
